In [18]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import pandas as pd

In [19]:
filenames = (glob.glob("data/spring17_by_major/*.csv"))

In [20]:
first = pd.read_csv(filenames[0])
first.head(5)

,Unnamed: 0,Semester Year Name Concat,Enrollment Cnt,Course Subject Short Nm,Course Number,Section Nbr,Crs Rpt College Schl Shrt Nm,Major
0,0,2017 Spring,3,African American Studies,5B,1,Clg of Letters & Science,african american studies
1,1,2017 Spring,1,African American Studies,11B,1,Clg of Letters & Science,african american studies
2,2,2017 Spring,1,African American Studies,28AC,1,Clg of Letters & Science,african american studies
3,3,2017 Spring,2,African American Studies,101,1,Clg of Letters & Science,african american studies
4,4,2017 Spring,1,African American Studies,115,1,Clg of Letters & Science,african american studies


In [21]:
def get_major_from_filename(filename):
    return filename.split("/")[-1].split(".")[0].replace("_", " ")

def get_filename_from_major(major):
    return "data/spring17_by_major/" + major.replace(" ", "_") + ".csv"

def add_major_to_csvs(filenames):
    for i in range(len(filenames)):
        filename = filenames[i]
        df = pd.read_csv(filename, index_col=None)
        df["Major"] = get_major_from_filename(filename)
        df.to_csv(filename, index=False)
    print("Added major column to csvs")

In [22]:
def building_single_df(filenames):
    agg_df = pd.read_csv(filenames[0])
    for i in range(len(filenames))[1:]:
        agg_df = agg_df.append(pd.read_csv(filenames[i]))
    return agg_df

def sanity_check(filenames):
    total = 0
    for i in range(len(filenames)):
        subtotal = sum(Table.read_table(filenames[i]).column("Enrollment Cnt"))
        print(filenames[i], subtotal)
        total += subtotal
    return total

In [23]:
agg_df = building_single_df(filenames)

In [24]:
agg_df.head(5)

,Unnamed: 0,Semester Year Name Concat,Enrollment Cnt,Course Subject Short Nm,Course Number,Section Nbr,Crs Rpt College Schl Shrt Nm,Major
0,0,2017 Spring,3,African American Studies,5B,1,Clg of Letters & Science,african american studies
1,1,2017 Spring,1,African American Studies,11B,1,Clg of Letters & Science,african american studies
2,2,2017 Spring,1,African American Studies,28AC,1,Clg of Letters & Science,african american studies
3,3,2017 Spring,2,African American Studies,101,1,Clg of Letters & Science,african american studies
4,4,2017 Spring,1,African American Studies,115,1,Clg of Letters & Science,african american studies


In [25]:
agg_prop = agg_df.groupby(['Course Subject Short Nm']).sum()
agg_prop['Total Prop'] = agg_prop['Enrollment Cnt']/sum(agg_prop['Enrollment Cnt'])
major_prop_all = (agg_prop.iloc[:, 3:]).sort_values("Total Prop", ascending=False).take(range(0, 25)).transpose()
major_prop_all # Description: Top 25 of the most enrolled in course subjects aka departments. 

Course Subject Short Nm,Computer Science,Mathematics,Business Admin-Undergrad,Economics,Chemistry,Psychology,Sociology,Physical Education,History,Physics,...,Music,Anthropology,Biology,Integrative Biology,Electrical Engineering,Public Health,Nutritional Science & Tox,Philosophy,Mechanical Engineering,Education
Total Prop,0.081479,0.06651,0.041211,0.038221,0.036741,0.036534,0.034039,0.033439,0.029153,0.027042,...,0.018447,0.017095,0.016088,0.015816,0.015808,0.01576,0.014417,0.01253,0.012354,0.010875


In [26]:
def major_prop(filename):
    major = get_major_from_filename(filename)
    major_df = pd.read_csv(filename)
    major_df = major_df.groupby(['Course Subject Short Nm']).sum()
    
    # Proportion of the <major> that enrolled in each respective department (Course Subject)
    major_df[major] = major_df['Enrollment Cnt']/sum(major_df['Enrollment Cnt'])
    major_df = (major_df.iloc[:, 3:]).transpose()
    return major_df

def build_major_props_df(top_num, total_prop_df): #top_num = top 80 => 80 
    majors = list(agg_df.groupby('Major')[['Enrollment Cnt']].sum().sort_values('Enrollment Cnt', ascending=False).take(range(top_num+1)).index)
    filenames = [get_filename_from_major(major) for major in majors if major != "ls undeclared"]
    rv_df = major_prop(filenames[0])
    for i in range(len(filenames))[1:]:
        inter_df = major_prop(filenames[i])
        rv_df = rv_df.append(inter_df)
    rv_df = rv_df[list(total_prop_df.columns)]
    rv_df = total_prop_df.append(rv_df).fillna(0)
    for column in total_prop_df:
        rv_df[column] = rv_df[column]/total_prop_df[column][0]
    return rv_df.iloc[1:, :]

In [27]:
major_prop_all_proto = build_major_props_df(80, major_prop_all)

In [28]:
major_prop_all = pd.DataFrame(columns=['Major', 'Department', 'Ratio'])
major = []
dept = []
ratio = []
for i in range(major_prop_all_proto.shape[0]):
    this = major_prop_all_proto.iloc[i, :]
    major += ([this.name] * len(list(this)))
    ratio += list(this)
    dept += list(this.index)
major_prop_all['Major'] = major
major_prop_all['Department'] = dept
major_prop_all['Ratio'] = ratio

### Building ordering to produce "diagonal" effect

In [ ]:
table = Table().from_df(major_prop_all)
tabledict = {}
for major in table.column('Major'):
    sub = table.where('Major', major)
    zipped = zip(sub.column('Department'), sub.column('Ratio'))
    maximum = max(sub.column('Ratio'))
    darkest = [d for d, r in zipped if r == maximum]
    tabledict[major] = [darkest[0], maximum]

order = Table(['order_Major', 'order_Department', 'order_Ratio'])
for key in tabledict:
    elem = tabledict[key]
    order = order.append([key, elem[0], elem[1]])
    
orderDept = order.group('order_Department').sort('count', descending=False).column('order_Department')

In [ ]:
orderdf = Table.to_df(order)
orderdf['Department'] = orderdf['order_Department'].map(dict(zip(orderDept, np.arange(0, len(orderDept)))))

orderMajor = list(orderdf.sort_values('Department')['order_Major'])

In [ ]:
major_prop_all['order_Department'] = major_prop_all['Department'].map(dict(zip(orderDept, np.arange(0, len(orderDept)))))
major_prop_all['order_Major'] = major_prop_all['Major'].map(dict(zip(orderMajor, np.arange(0, len(orderMajor)))))

result = Table().from_df(major_prop_all).groups(['order_Department', 'order_Major'], lambda x: x[0])